<h1><b>Will You Survive On Titanic AI</b></h1>
<h2>Importing Libraries and Database</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns

In [2]:
titanic = pd.read_csv(r"titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
# sns.pairplot(titanic, hue="Survived")

<h1>Data Cleanup</h1>

In [4]:
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket'])
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,0,3,male,34.5,0,0,7.8292
1,893,1,3,female,47.0,1,0,7.0000
2,894,0,2,male,62.0,0,0,9.6875
3,895,0,3,male,27.0,0,0,8.6625
4,896,1,3,female,22.0,1,1,12.2875


In [5]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row,column]-min1)/(max1-min1)
                x.at[row,column]=xprim 
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [6]:
DataProcessing.shuffle(titanic)

for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)

        
# values=titanic.select_dtypes(exclude="object")
# columnNames=values.columns.tolist()
# for column in columnNames:
# column = "PassengerId"
# for row in range(len(titanic)-1,-1,-1):
#     print(titanic.at[row,column])
#     temp = titanic.at[row,column]
#     if isinstance(temp, (int, float, complex)):
#         print(titanic.at[row,column])
        
        
#     else:
#         print(titanic.iloc[row])
#         titanic = titanic.drop(row,axis=0)


titanic.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1104,0,2,1,17.0,0,0,73.5000
1,895,0,3,1,27.0,0,0,8.6625
2,1271,0,3,1,5.0,4,2,31.3875
3,1307,0,3,1,38.5,0,0,7.2500
4,944,1,2,0,20.0,2,1,23.0000


In [7]:
DataProcessing.normalization(titanic)
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.508393,0,0.5,1,0.221944,0.00,0.000000,0.143462
1,0.007194,0,1.0,1,0.353818,0.00,0.000000,0.016908
2,0.908873,0,1.0,1,0.063695,0.50,0.222222,0.061264
3,0.995204,0,1.0,1,0.505473,0.00,0.000000,0.014151
4,0.124700,1,0.5,0,0.261506,0.25,0.111111,0.044893


In [8]:
titanic = titanic.dropna()
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.508393,0,0.5,1,0.221944,0.00,0.000000,0.143462
1,0.007194,0,1.0,1,0.353818,0.00,0.000000,0.016908
2,0.908873,0,1.0,1,0.063695,0.50,0.222222,0.061264
3,0.995204,0,1.0,1,0.505473,0.00,0.000000,0.014151
4,0.124700,1,0.5,0,0.261506,0.25,0.111111,0.044893


In [9]:
trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

In [10]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.508393,0,0.5,1,0.221944,0.00,0.000000,0.143462
1,0.007194,0,1.0,1,0.353818,0.00,0.000000,0.016908
2,0.908873,0,1.0,1,0.063695,0.50,0.222222,0.061264
3,0.995204,0,1.0,1,0.505473,0.00,0.000000,0.014151
4,0.124700,1,0.5,0,0.261506,0.25,0.111111,0.044893


<h2>k-nearest neighbors algorithm</h2>

In [15]:
class Knn:
    @staticmethod
    def knn(x,element,k=1):
        dist = []
        y = 0
        for i in range(len(x)):
            sum = 0
            for j in range(len(element)-1):
                sum += (x.iloc[i,j] - element.iloc[j])**2
            dist.append(sum**(1/2))
        x['dist']=dist
        y = x.sort_values("dist")
        kElementów = {}
        for i in range(k):
            kElementów[y["Survived"].iat[i]] = 0
        for i in range(k):
            kElementów[y["Survived"].iat[i]] +=1
        return max(kElementów, key=kElementów.get)
    @staticmethod
    def accuracy(x,z,k):
        good = 0
        for i in range(len(z)):
            if( Knn.knn(x,z.iloc[i],k) == z.iloc[i][4] ):
                good +=1
        return good/z.shape[0]

In [16]:
Knn.accuracy(trainingSet,validatingSet,4)

C:\Users\rafal\AppData\Local\Temp\ipykernel_24072\975701900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['dist']=dist


0.0

In [23]:

Knn.knn(trainingSet,validatingSet.iloc[3],3)

C:\Users\rafal\AppData\Local\Temp\ipykernel_24072\975701900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['dist']=dist


1

In [20]:
validatingSet.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
292,0.971223,0,0.5,1,0.261506,0.000,0.0,0.027058
293,0.398082,0,0.0,1,0.630753,0.000,0.0,0.098561
294,0.606715,0,1.0,1,0.314256,0.000,0.0,0.018201
295,0.285372,1,0.5,0,0.380193,0.125,0.0,0.050749
297,0.573141,1,0.0,0,0.630753,0.125,0.0,0.207728
